# Machine Learning

Crearemos un algoritmo compare la cantidad de casos con las temperaturas medias cuatrimestrales a nivel nacional para determinar la relación de las mismas a través de una regresión lineal

In [1]:
# Importo las librerías necesarias
import pandas as pd
import numpy as np
import re
import calendar
import scipy as sp
import sklearn 
import seaborn as sns
import matplotlib.pyplot as plt
import pycaret as pc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
sns.set_theme()

In [2]:
# Ahora leo las tablas creadas por las transformaciones anteriores
TablaCompleta = pd.read_csv(r'../Tablas_creadas/TablaCompleta.csv')
Reportes = pd.read_csv(r'../Tablas_creadas/Reportes.csv')
Estaciones = pd.read_csv(r'../Tablas_creadas/Estaciones.csv')
Temp_humedad = pd.read_csv(r'../Tablas_creadas/Temp_humedad.csv')

Preproceso la data, calculando:

    1- Temperatura promedio por cuatrimestre
    2- Casos por cuatrimestre  
Luego uno esa data en un nuevo dataframe con las columnas: Cuatrimestre | Casos | Temperatura
 

In [3]:
# Formo el dataframe nuevo:
# ! Temperaturas promedio por cuatrimestre:
def month_name_to_number(month_name):
    month_abbr = month_name[:3].capitalize()
    return list(calendar.month_abbr).index(month_abbr) 

Meses_temp = Temp_humedad['mes'].apply(month_name_to_number)

# Le cambio el formato a la columna meses
Meses_temp = pd.to_datetime(Meses_temp, format='%m').dt.month

# Me aseguro que figure como series para poder hacer los siguientes pasos
Meses_temp = pd.Series(Meses_temp)

# Nuevo df con los datos que necesito
df_temp = pd.DataFrame({'Mes': Meses_temp, 'Año': Temp_humedad['año']})

# Vuelvo a tirar el format %m por si acaso
df_temp['Mes'] = pd.to_datetime(df_temp['Mes'], format='%m').dt.month


# Hago una columna para los cuatrimestres
df_temp['Cuatrimestre'] = df_temp['Mes'].apply(lambda x: (x - 1) // 3 + 1)

# Hago otra que incluye columna y año en un formato donde es facilmente ordenable
df_temp['Cuatrimestre Completo'] = df_temp.apply(lambda row: f"{row['Año']} Q{(row['Mes'] - 1) // 3 + 1}", axis=1)

df_temp['Temperatura Media'] = Temp_humedad['T Media'].astype(float)

# Saco la temperatura media cuatrimestral

temp_sumatorias = df_temp.groupby('Cuatrimestre Completo')['Temperatura Media'].median().reset_index()

# ! Casos por Cuatrimestre
Meses_reportes = Reportes['mes'].apply(month_name_to_number)


df_reportes = pd.DataFrame({'Mes': Meses_reportes, 'Año': Reportes['año']})

df_reportes['Cantidad'] = Reportes['cantidad'].astype(int)

# Hago una columna para los cuatrimestres
df_reportes['Cuatrimestre'] = df_reportes['Mes'].apply(lambda x: (x - 1) // 3 + 1)

# Hago otra que incluye columna y año en un formato donde es facilmente ordenable
df_reportes['Cuatrimestre Completo'] = df_reportes.apply(lambda row: f"{row['Año']} Q{(row['Mes'] - 1) // 3 + 1}", axis=1)

# Agrupo los casos por cuatrimestre

cuatrimestre_sumatorias = df_reportes.groupby('Cuatrimestre Completo')['Cantidad'].sum().reset_index()

# ! Concateno los nuevos datos en 1 dataframe nuevo:

df_ML = pd.merge(temp_sumatorias, cuatrimestre_sumatorias, how= 'outer')


# filleo los NaN por 0 para los cuatrimestres sin casos:

df_ML = df_ML.fillna(0)


In [4]:
print(df_ML)

   Cuatrimestre Completo  Temperatura Media  Cantidad
0                2018 Q1               23.1     271.0
1                2018 Q2               13.2    2507.0
2                2018 Q3               10.8      19.0
3                2018 Q4               20.2       7.0
4                2019 Q1               21.9     938.0
5                2019 Q2               13.8    1796.0
6                2019 Q3               10.9      29.0
7                2019 Q4               21.0      26.0
8                2020 Q1               23.4   16144.0
9                2020 Q2               13.4   38463.0
10               2020 Q3               10.8     407.0
11               2020 Q4               21.3      46.0
12               2021 Q1               22.3    1304.0
13               2021 Q2               13.1    2684.0
14               2021 Q3               11.9       3.0
15               2021 Q4               21.2       0.0
16               2022 Q1               22.2     148.0
17               2022 Q2    

Separo la data en X (temperatura promedio por cuatrimestre) e y (casos)

El siguiente paso es ya crear el modelo de regresión lineal:

In [5]:
# Split the data into features (X) and target (y)
X = df_ML['Temperatura Media'].values.reshape(-1, 1)
y = df_ML['Cantidad'].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create polynomial features
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_poly)


In [6]:
# evalúo el desempeño del modelo

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 9432533117.31
R-squared: -0.37
